In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import shutil
tujuan = '/content/'
sumber = '/content/drive/MyDrive/all_model.zip'
shutil.copy(sumber, tujuan)

'/content/all_model.zip'

In [8]:
!unzip all_model.zip

Archive:  all_model.zip
  inflating: decoder.h5              
  inflating: symptom_tokenizer.pkl   
  inflating: caption_tokenizer.pkl   
  inflating: encoder.h5              
  inflating: unet_model_baru.pth     
  inflating: inceptionv3_best_val_loss.h5  


In [9]:
!pip install -q segmentation-models-pytorch albumentations gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.6 MB/s eta 0:00:00


In [10]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
from PIL import Image
import cv2
import torch
import torchvision.transforms as transforms
import segmentation_models_pytorch as smp

inception_model = load_model('/content/inceptionv3_best_val_loss.h5')
encoder_model = load_model('/content/encoder.h5')
decoder_model = load_model('/content/decoder.h5')

with open('/content/symptom_tokenizer.pkl', 'rb') as f:
    symptom_tokenizer = pickle.load(f)
with open('/content/caption_tokenizer.pkl', 'rb') as f:
    caption_tokenizer = pickle.load(f)

symptom_labels = ['sehat', 'berkerak', 'cekung', 'bintik', 'hitam', 'cokelat']
max_caption_len = 20


device = torch.device("cpu")

unet_model = smp.Unet(
    encoder_name="vgg16_bn",
    encoder_weights='imagenet',
    in_channels=3,
    classes=3,
)
unet_model.load_state_dict(torch.load('/content/unet_model_baru.pth', map_location=device))
unet_model.to(device)
unet_model.eval()

#inceptionV3
def preprocess_image(img):
    img = img.resize((299, 299))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.inception_v3.preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)


def preprocess_for_unet(img):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    return transform(img).unsqueeze(0).to(device)


def segment_image(img):
    original_img = img.resize((256, 256)).convert("RGB")
    img_tensor = preprocess_for_unet(original_img)

    with torch.no_grad():
        output = unet_model(img_tensor)
    mask = torch.argmax(output.squeeze(), dim=0).cpu().numpy()

    class1_mask = (mask == 1).astype(np.uint8)

    contours, _ = cv2.findContours(class1_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    overlay = np.array(original_img)
    overlay = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)

    cv2.drawContours(overlay, contours, -1, (0, 255, 0), thickness=1)

    overlay_rgb = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
    return Image.fromarray(overlay_rgb)


def predict_symptoms(img_array):
    probs = inception_model.predict(img_array)[0]
    selected = [symptom_labels[i] for i, p in enumerate(probs) if p > 0.5]
    return selected

def generate_caption(symptoms):
    sequence = symptom_tokenizer.texts_to_sequences([' '.join(symptoms)])
    sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=5, padding='post')

    states = encoder_model.predict(sequence)

    target_seq = np.array([[caption_tokenizer.word_index['startseq']]])
    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = caption_tokenizer.index_word.get(sampled_token_index, '')

        if sampled_word == 'endseq' or len(decoded_sentence) >= max_caption_len:
            stop_condition = True
        else:
            decoded_sentence.append(sampled_word)

        target_seq = np.array([[sampled_token_index]])
        states = [h, c]

    return ' '.join(decoded_sentence)

def process_image(image):
    img_array = preprocess_image(image)
    symptoms = predict_symptoms(img_array)
    if not symptoms:
        caption = "penyakit tidak terdeteksi"
    else:
        caption = generate_caption(symptoms)
    segmentation = segment_image(image)
    return caption, segmentation

interface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Caption"),
        gr.Image(type="pil", label="Segmentasi")
    ],
    title="Image Captioning Penyakit Buah Jeruk"
)

interface.launch(debug=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/554M [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://809e981ecb4efdcbf0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://809e981ecb4efdcbf0.gradio.live
